In [1]:
from src.MOF_builder.vlx_mof_builder_v2 import mof_builder
mof = mof_builder()
mof.preparation.select_mof_family('UiO-67')
mof.preparation.select_node_metal('Zr')
#mof.preparation.use_dummy_node(True)
mof.preparation.fetch_node()
mof.preparation.fetch_linker('ndi.xyz')
mof.preparation_check()
mof.rotation_optimizer_maxfun = 1000
mof.use_saved_optimized_rotations_npy('rota')
#save optimized rotations to numpy file for later use
mof.save_optimized_rotations('rota')
mof.set_supercell([1,1,1])
mof.build()
mof.write_gro()
mof.set_remove_edge_list([30,31,32])
mof.set_remove_node_list([])
mof.make_defects_missing()
mof.write_defect_gro()


Available MOF Family:
TEST-MOF UiO-66 UiO-67 UiO-68 PCN-222 MOF-545 NU-1000 PCN-224 MOF-525 MOF-535 MOF-808 MOF-818 PCN-777 NU-901 MIL-100 MIL-101 PCN-228 PCN-229 PCN-230
MOF builder is initialized, please prepare the building material by calling the preparation driver
1.preparation.select_mof_family()
2.preparation.select_node_metal()
3.preparation.use_dummy_node()
4.preparation.fetch_node()
5.preparation.fetch_linker()
***check the preparation status by calling preparation_check()***
UiO-67 selected
available metal nodes: ['Zr', 'Hf']
please select a metal node
will search template cif files in /Users/chenxili/GitHub/vlx_mofbuilder/data/template_database
fcu.cif is found in template_database
fcu.cif will be used for MOF building
Zr node is selected
if you want to use dummy node, please select dummy node type with select_dummy_node method
now we are going to fetch the node pdb file
found the file including ['12c', 'Zr']
File copied from /Users/chenxili/GitHub/vlx_mofbuilder/data/nodes

## defects and postprocess of node residues split 
 done: make defects on eG and add terminations 
#### todo: add linker exchange defects by superimpose the x ;linker_termination can be realized by linker exchange;
#### todo: split dummy nodes by use dummy dict info ;

## todo: multitopic test
## todo: bridge node and virtual edge test 